# Figure 5

NOTE: If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/Aperiodic-Exponent-Model.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/Aperiodic-Exponent-Model')

---

## Load packages

In [ ]:
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from   matplotlib.backends.backend_pdf import PdfPages

## Load functions for computations and plotting

In [ ]:
from one_over_f_functions import *

## Define the model

In [ ]:
# Simulate the generative model.
def predator_prey(noise,params):
    
    N = 250000
    dt = 0.0002
    alpha = params["alpha"]
    gamma = params["gamma"]
    
    x = np.zeros([N])                         # Output variables
    y = np.zeros([N])
    t = np.arange(N)*dt                       # Time [a.u.]
    
    x[0] = 0.2+0.2*np.random.rand()           # Initial conditions
    y[0] = 0.2+0.2*np.random.rand()
    
    for n in np.arange(N-1):                  # Simulate the model
        x[n+1] = x[n] + dt*(x[n]*(gamma-x[n])/gamma - x[n]*y[n] + noise["x"]*np.random.randn())
        y[n+1] = y[n] + dt*(-alpha*y[n]             + x[n]*y[n] + noise["y"]*np.random.randn())
        
    x = x[50000:]
    y = y[50000:]
    t = t[50000:]-t[50000]
    
    return x,y,t

## Figure 5A: Plot analytic spectra for the predator-prey model.

In [ ]:
noise = {"x":[],                           # Set the prey noise,
         "y":1  }                          # ... and the predator noise.
params= {"alpha":0.25,                     # Set the parameters.
         "gamma":0.6}

def S11_analytic(omega, noise_x):          # Compute spectrum of predator-prey model using analytic result.
    a = -params["alpha"]/params["gamma"]
    b = -params["alpha"]
    c = 1-params["alpha"]/params["gamma"]
    d = 0
    B11 = noise_x
    B12 = 0
    B21 = 0
    B22 = noise["y"]                       # See manuscript Appendix, Equation (A3)
    S11 = (0.0625 + B11**2 * omega**2) / (2*np.pi*(0.0212674 - 0.118056*omega**2 + omega**4))
    return S11
                                           # Plot the spectrum for different prey noise.
omega = np.arange(0.1,10000,0.1); f = 2*np.pi*omega;
fig = plt.figure(figsize=(5, 5), dpi=80)
S11 = S11_analytic(omega,0.1);   l1 = plt.plot(np.log10(omega), np.log10(S11), label="$\sigma_x$=0.1")
S11 = S11_analytic(omega,0.01);  l2 = plt.plot(np.log10(omega), np.log10(S11), label="$\sigma_x$=0.01")
S11 = S11_analytic(omega,0.001); l3 = plt.plot(np.log10(omega), np.log10(S11), label="$\sigma_x$=0.001")
S11 = S11_analytic(omega,0.0);   l4 = plt.plot(np.log10(omega), np.log10(S11), label="$\sigma_x$=0.0")
plt.xlabel("Log$_{10}$($\omega$)")
plt.ylabel("Log$_{10}$(P)")
plt.yticks(np.arange(-18, 0, 2))
plt.grid("on")
plt.legend();

#plt.savefig("./PDFs/Figure-5A.pdf", bbox_inches='tight')

## Figure 5B: Plot estimated spectra from simualted predator-prey data.

In [ ]:
noise = {"x":[],                           # Set the prey noise,
         "y":1  }                          # ... and the predator noise.
params= {"alpha":0.25,                     # Set the parameters.
         "gamma":0.6}

prey_noise = [0.1, 0.01, 0.001, 0.0];

fig = plt.figure(figsize=(5, 5), dpi=80)
for cnt, noise_x in enumerate(prey_noise):
    noise["x"]=noise_x;                            # Prey noise = 0.1.
    [x,y,t] = predator_prey(noise,params)      # Simulate model,
    [S,f]   = compute_spectrum(x,t)            # ... compute the spectrum.
    plt.plot(np.log10(f[2:]), np.log10(S[2:]), label="$\sigma_x$="+str(noise_x));
plt.xlabel("Log$_{10}$(Frequency [Hz]")
plt.ylabel("Log$_{10}$(P)")
plt.xlim([0,3])
plt.ylim([-24,-6])
plt.yticks(np.arange(-24, -6, 2))
plt.grid("on")
plt.legend();

#plt.savefig("./PDFs/Figure-5B.pdf", bbox_inches='tight')